In [1]:
import os
import re
import pandas as pd
import numpy as np

from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo


import requests
from LxmlSoup import LxmlSoup
from bs4 import BeautifulSoup
import time
import random

In [2]:
# ---------- 1) Сбор (парсинг страниц) с HomesOverseas ----------

def parse_homesoverseas(country_url_part: str, has_info: bool = True, max_pages: int = 20) -> pd.DataFrame:
    """
    Скачивает карточки объявлений о недвижимости с сайта HomesOverseas
    для конкретной страны и возвращает «сырые» данные в виде DataFrame.

    Параметры:
      country_url_part : фрагмент URL страны (например, 'Austria')
      has_info         : у части стран есть дополнительный блок «info»; если False — игнорируем его
      max_pages        : ограничение на количество страниц, которые парсим (страницы идут p=0,1,2,...)

    Возвращает:
      DataFrame со столбцами:
        - 'name'  : название/описание объявления
        - 'price' : цена (как текст, без чистки)
        - 'info'  : дополнительная информация (если has_info=True)
    """
    all_names, all_prices, all_infos = [], [], []
    page = 0

    # крутимся по страницам, пока сайт отдаёт данные и не достигнут max_pages
    while True:
        # пример URL: https://www.homesoverseas.ru/country/Austria/apartments?p=0&s=50
        url = f"https://www.homesoverseas.ru/country/{country_url_part}/apartments?p={page}&s=50"
        print(f"[{country_url_part}] p={page}: {url}")

        # базовый GET-запрос; при не-200 коде завершаем цикл
        resp = requests.get(url)
        if resp.status_code != 200:
            print(f"HTTP {resp.status_code}, стоп.")
            break

        # парсим HTML: ищем контейнеры с названием и ценой
        soup = LxmlSoup(resp.text)
        names  = soup.find_all("div", class_="name_mess")  # блок с заголовком/названием
        prices = soup.find_all("div", class_="price_tr")   # блок с ценой

        # если страница пустая (нет и названий, и цен) — значит дальше контента нет
        if len(names) == 0 and len(prices) == 0:
            print("Пустая страница => завершение.")
            break

        if has_info:
            # для стран с доп.инфо — собираем третий список
            infos = soup.find_all("div", class_="line mrg5T options")
            # на всякий случай берём минимальную длину, чтобы не словить IndexError
            length = min(len(names), len(prices), len(infos))
            for i in range(length):
                all_names.append(names[i].text())
                all_prices.append(prices[i].text())
                all_infos.append(infos[i].text())
        else:
            # если доп.инфо нет — сохраняем только названия и цены
            length = min(len(names), len(prices))
            for i in range(length):
                all_names.append(names[i].text())
                all_prices.append(prices[i].text())

        # переходим к следующей странице
        page += 1
        if page >= max_pages:
            print(f"max_pages={max_pages} => stop.")
            break

        # (необязательно) небольшая задержка — вежливость к сайту
        time.sleep(0.5)

    # собираем итоговый DataFrame
    if has_info:
        df = pd.DataFrame({"name": all_names, "price": all_prices, "info": all_infos})
    else:
        df = pd.DataFrame({"name": all_names, "price": all_prices})

    return df


# ---------- 2) Преобразование (очистка и расчёты) ----------

def transform_homesoverseas_data(df: pd.DataFrame, has_info: bool = True) -> pd.DataFrame:
    """
    Приводит «сырые» данные к единому табличному виду:
    - чистит пробелы/мусор,
    - превращает цену в число,
    - по info (если есть) пытается извлечь площадь/кол-во комнат,
    - считает «цену за м²»,
    - переименовывает колонки и упорядочивает их.
    """
    df = df.copy()

    # 2.1 Нормализуем пробелы/переводы строк в строковых колонках
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = (
                df[c]
                .astype(str)
                .str.replace(r"\s+", " ", regex=True)  # множественные пробелы → один
                .str.strip()
            )

    # 2.2 Функция очистки цены: убираем «от », валюту/мусор, меняем , на . и конвертируем в float
    def clean_price(p):
        if not isinstance(p, str):
            return np.nan
        p = p.replace("от ", "")
        if "ценапозапросу" in p.lower():  # если «цена по запросу» — пропускаем
            return np.nan
        p = p.replace("€", "").replace("\xa0", "")  # евро-символ и неразрывные пробелы
        p = re.sub(r"[^0-9.,]", "", p).replace(",", ".").strip()  # оставляем только цифры/точку/запятую
        if p == "":
            return np.nan
        try:
            return float(p)
        except:
            return np.nan

    # применяем чистку к столбцу price и выкидываем пустые цены
    df["price"] = df["price"].apply(clean_price)
    df.dropna(subset=["price"], inplace=True)

    # 2.3 Инициализируем площадь/комнаты NaN (заполним, если есть info)
    df["square"] = np.nan
    df["rooms"] = np.nan

    if has_info and "info" in df.columns:
        # извлекаем площадь — либо по слову «площадь ...», либо шаблоном «NN м»
        def extract_square(txt: str):
            m = re.search(r"площадь\s+([\d.,]+)", txt, re.IGNORECASE)
            if m:
                val = m.group(1).replace(",", ".")
                try:
                    return float(val)
                except:
                    pass
            m2 = re.search(r"([\d.,]+)\s*м", txt)  # например, «45 м»
            if m2:
                val = m2.group(1).replace(",", ".")
                try:
                    return float(val)
                except:
                    pass
            return np.nan

        # извлекаем кол-во комнат — по слову «спальня/спален»
        def extract_rooms(txt: str):
            m = re.search(r"(\d+)\s*спал", txt, re.IGNORECASE)
            if m:
                return float(m.group(1))
            return np.nan

        df["square"] = df["info"].apply(extract_square)
        df["rooms"] = df["info"].apply(extract_rooms)

    # 2.4 Считаем цену за квадратный метр (если известны цена и положительная площадь)
    df["price_per_sq"] = np.where(
        (df["price"].notna()) & (df["square"] > 0),
        df["price"] / df["square"],
        np.nan
    )

    # 2.5 Добавляем сквозную нумерацию
    df.insert(0, "№", range(1, len(df) + 1))

    # 2.6 Приводим к финальным названиям колонок (для единообразия между сайтами)
    rename_map = {
        "name": "Название",
        "price": "Цена",
        "square": "Площадь (м²)",
        "rooms": "Комнаты",
        "price_per_sq": "Цена за м²"
    }
    df = df.rename(columns=rename_map)

    # 2.7 Оставляем только нужные колонки (если каких-то нет — просто их не будет)
    wanted = ["№", "Название", "Цена", "Площадь (м²)", "Комнаты", "Цена за м²"]
    exist = [c for c in wanted if c in df.columns]
    df = df[exist]

    return df


# ---------- 3) Сохранение в Excel (форматированная таблица + мини-сводная) ----------

def save_homesoverseas_excel(df: pd.DataFrame, file_path: str) -> None:
    """
    Сохраняет DataFrame в Excel в виде «умной таблицы»:
      - заголовки начиная со второй строки,
      - данные с третьей,
      - включена строка итогов (СРЕДНЕЕ для цены/цены за м²),
      - добавлена мини-сводная (H1..I2) со средними значениями.

    Если df пустой — просто сохраняет пустой Excel (для единообразия пайплайна).
    """
    wb = Workbook()
    ws = wb.active
    ws.title = "Data"

    if df.empty:
        # сохраняем просто как плоскую таблицу — чтобы на диске всё равно появился файл
        df.to_excel(file_path, index=False)
        wb.save(file_path)
        return

    cols = df.columns.to_list()
    nrows = len(df)
    ncols = len(cols)

    # 1) Заголовки в строке 2
    header_row = 2
    for j, c_name in enumerate(cols):
        ws.cell(row=header_row, column=j + 1, value=c_name)

    # 2) Данные со строки 3
    data_start = 3
    for i in range(nrows):
        for j, c_name in enumerate(cols):
            ws.cell(row=data_start + i, column=j + 1, value=df.iat[i, j])

    # 3) Диапазон таблицы: A2 .. (последний столбец)(2 + nrows)
    data_end_row = data_start + nrows - 1  # = 2 + nrows
    ref_range = f"A{header_row}:{get_column_letter(ncols)}{data_end_row}"

    tab = Table(displayName="Table1", ref=ref_range)
    style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True, showColumnStripes=False)
    tab.tableStyleInfo = style
    tab.totalsRowShown = True  # показываем строку итогов

    # 4) Настройка итогов: в колонке «Название» — подпись «СРЕДНЕЕ:»,
    #    в колонках «Цена» и «Цена за м²» — функция average
    for i, colObj in enumerate(tab.tableColumns):
        hdr = cols[i]
        if hdr == "Название":
            colObj.totalsRowLabel = "СРЕДНЕЕ:"
            colObj.totalsRowFunction = None
        elif hdr in ["Цена", "Цена за м²"]:
            colObj.totalsRowFunction = "average"
        else:
            colObj.totalsRowFunction = None

    ws.add_table(tab)

    # 5) Немного формата: ширина столбцов и формат валюты
    for j, c_name in enumerate(cols, start=1):
        ws.column_dimensions[get_column_letter(j)].width = 15
        if c_name in ["Цена", "Цена за м²"]:
            for row_i in range(data_start, data_end_row + 1):
                ws.cell(row=row_i, column=j).number_format = '#,##0.00 "€"'

    # 6) Мини-сводная (средние) в H1..I2
    ws["H1"] = "Средняя цена"
    ws["I1"] = "Средняя цена за м²"
    ws["H2"] = f"=SUBTOTAL(101,Table1[[#All],[Цена]])"
    ws["I2"] = f"=SUBTOTAL(101,Table1[[#All],[Цена за м²]])"
    ws["H2"].number_format = '#,##0.00 "€"'
    ws["I2"].number_format = '#,##0.00 "€"'
    ws.column_dimensions["H"].width = 20
    ws.column_dimensions["I"].width = 20

    wb.save(file_path)


# ---------- 4) Пример запуска по нескольким странам (только HomesOverseas) ----------

def main():
    """
    Прогон по набору стран (HomesOverseas) с сохранением отдельных Excel-файлов.
    """
    countries = [
        ("Австралия", "Avstralija", False),
        ("Австрия", "Austria", True),
        ("Андорра", "Andorra", True),
        ("Болгария", "Bolgarija", True),
        ("Венгрия", "Vengrija", True),
        ("Германия", "Germanija", True),
        ("Греция", "Grecija", True),
        ("Грузия", "Gruzija", True),
        ("Израиль", "Izrail", True),
        ("Испания", "Spain", True),
        ("Италия", "Italija", True),
        ("Канада", "Kanada", True),
        ("Кипр", "Kipr", True),
        ("Латвия", "Latvija", True),
        ("Люксембург", "Luxemburg", True),
        ("Монако", "Monako", True),
        ("ОАЭ", "OAE", True),
        ("Португалия", "Portugalija", True),
        ("Словакия", "Slovakija", True),
        ("Великобритания", "Velikobritanija", True),
        ("США", "SShA", True),
        ("Таиланд", "Tailand", True),
        ("Турция", "Turcija", True),
        ("Финляндия", "Finljandija", True),
        ("Франция", "Francija", True),
        ("Черногория", "Chernogorija", True),
        ("Швейцария", "Shvejcarija", True),
        ("Эстония", "Estonija", True),
    ]

    base_path = r"C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)"

    for (country_rus, url_part, info_flag) in countries:
        print(f"\n=== HomesOverseas: {country_rus} => {url_part} ===")
        df_raw = parse_homesoverseas(url_part, has_info=info_flag, max_pages=20)
        df_clean = transform_homesoverseas_data(df_raw, has_info=info_flag)

        file_path = rf"{base_path}\{country_rus}.xlsx"
        save_homesoverseas_excel(df_clean, file_path)
        print(f"[{country_rus}] => {file_path}")

        
if __name__=="__main__":
    main()


=== HomesOverseas: Австралия => Avstralija ===
[Avstralija] p=0: https://www.homesoverseas.ru/country/Avstralija/apartments?p=0&s=50
Пустая страница => завершение.
[Австралия] => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Австралия.xlsx

=== HomesOverseas: Австрия => Austria ===
[Austria] p=0: https://www.homesoverseas.ru/country/Austria/apartments?p=0&s=50
[Austria] p=1: https://www.homesoverseas.ru/country/Austria/apartments?p=1&s=50
[Austria] p=2: https://www.homesoverseas.ru/country/Austria/apartments?p=2&s=50
Пустая страница => завершение.
[Австрия] => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Австрия.xlsx

=== HomesOverseas: Андорра => Andorra ===
[Andorra] p=0: https://www.homesoverseas.ru/country/Andorra/apartments?p=0&s=50
[Andorra] p=1: https://www.homesoverseas.ru/country/Andorra/apartments?p=1&s=50
Пустая страница => завершение.
[Андорра] => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Андорра.xlsx

=== HomesOverseas: Болгария => Bolgarija ===
[Bo

[Italija] p=13: https://www.homesoverseas.ru/country/Italija/apartments?p=13&s=50
[Italija] p=14: https://www.homesoverseas.ru/country/Italija/apartments?p=14&s=50
Пустая страница => завершение.
[Италия] => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Италия.xlsx

=== HomesOverseas: Канада => Kanada ===
[Kanada] p=0: https://www.homesoverseas.ru/country/Kanada/apartments?p=0&s=50
Пустая страница => завершение.
[Канада] => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Канада.xlsx

=== HomesOverseas: Кипр => Kipr ===
[Kipr] p=0: https://www.homesoverseas.ru/country/Kipr/apartments?p=0&s=50
[Kipr] p=1: https://www.homesoverseas.ru/country/Kipr/apartments?p=1&s=50
[Kipr] p=2: https://www.homesoverseas.ru/country/Kipr/apartments?p=2&s=50
[Kipr] p=3: https://www.homesoverseas.ru/country/Kipr/apartments?p=3&s=50
[Kipr] p=4: https://www.homesoverseas.ru/country/Kipr/apartments?p=4&s=50
[Kipr] p=5: https://www.homesoverseas.ru/country/Kipr/apartments?p=5&s=50
[Kipr] p=6: https://ww

[Tailand] p=7: https://www.homesoverseas.ru/country/Tailand/apartments?p=7&s=50
[Tailand] p=8: https://www.homesoverseas.ru/country/Tailand/apartments?p=8&s=50
[Tailand] p=9: https://www.homesoverseas.ru/country/Tailand/apartments?p=9&s=50
[Tailand] p=10: https://www.homesoverseas.ru/country/Tailand/apartments?p=10&s=50
[Tailand] p=11: https://www.homesoverseas.ru/country/Tailand/apartments?p=11&s=50
[Tailand] p=12: https://www.homesoverseas.ru/country/Tailand/apartments?p=12&s=50
[Tailand] p=13: https://www.homesoverseas.ru/country/Tailand/apartments?p=13&s=50
[Tailand] p=14: https://www.homesoverseas.ru/country/Tailand/apartments?p=14&s=50
[Tailand] p=15: https://www.homesoverseas.ru/country/Tailand/apartments?p=15&s=50
[Tailand] p=16: https://www.homesoverseas.ru/country/Tailand/apartments?p=16&s=50
[Tailand] p=17: https://www.homesoverseas.ru/country/Tailand/apartments?p=17&s=50
[Tailand] p=18: https://www.homesoverseas.ru/country/Tailand/apartments?p=18&s=50
[Tailand] p=19: https:

In [3]:
# ---------- 1) Сбор (парсинг страниц) с Tranio ----------

def parse_tranio_country(country_url_part: str, max_pages: int = 20) -> pd.DataFrame:
    """
    Скачивает карточки объявлений о недвижимости со страницы страны на Tranio
    и возвращает «сырые» данные (title/price/info) в виде DataFrame.

    Параметры:
      country_url_part : фрагмент URL страны (например, 'spain', 'italy')
      max_pages        : верхний лимит количества страниц (1..max_pages)

    Пагинация у Tranio:
      - первая страница:  ?order=rank
      - последующие:     ?order=rank&page=2,3,...

    Возвращает:
      DataFrame со столбцами ['name', 'price', 'info'] (как текст).
    """
    unique_entries = set()   # используем для отсечения дублей между страницами
    page = 1

    while True:
        # собираем URL для текущей страницы
        if page == 1:
            url = f"https://tranio.ru/{country_url_part}/apartments/?order=rank"
        else:
            url = f"https://tranio.ru/{country_url_part}/apartments/?order=rank&page={page}"

        print(f"[Tranio/{country_url_part}] Парсим страницу {page}: {url}")
        resp = requests.get(url)
        if resp.status_code != 200:
            # если сайт не отдал 200 — прекращаем
            print(f"Ответ {resp.status_code}. Останавливаемся.")
            break

        # парсим HTML, находим контейнеры карточек
        soup = BeautifulSoup(resp.text, 'html.parser')
        snippets = soup.find_all("div", class_="snippet slide")  # родительский блок карточки

        if not snippets:
            # когда карточек нет, можно останавливать пагинацию
            print("Нет объявлений на странице. Останавливаемся.")
            break

        found_new = False  # были ли новые карточки на этой странице
        for snippet in snippets:
            # внутренние элементы карточки: заголовок/цена/краткие фичи
            title_el = snippet.find("div", class_="snippet-title")
            price_el = snippet.find("div", class_="snippet-price")
            info_el  = snippet.find("div", class_="snippet-features")

            l_name  = title_el.get_text(strip=True) if title_el else "No name"
            l_price = price_el.get_text(strip=True) if price_el else "No price"
            l_info  = info_el.get_text(strip=True)  if info_el  else "No info"

            entry = (l_name, l_price, l_info)
            if entry not in unique_entries:
                # сохраняем только новые записи
                unique_entries.add(entry)
                found_new = True

        if not found_new:
            # если на странице вообще не было новых карточек — вероятно, дублируем предыдущую
            print(f"Страница {page} полностью дублирует предыдущие. Остановка.")
            break

        page += 1
        if page > max_pages:
            print(f"Достигнут лимит max_pages={max_pages}. Остановка.")
            break

    # переводим множество в DataFrame
    data = list(unique_entries)
    df = pd.DataFrame(data, columns=["name", "price", "info"])
    return df


# ---------- 2) Преобразование (очистка и расчёты) ----------

def transform_tranio_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Приводит «сырые» данные Tranio к единому виду:
      - фильтрует жилые комплексы (по ключевым словам в info),
      - чистит info от лишних слов/символов,
      - извлекает площадь/комнаты,
      - нормализует стоимость в евро (€, $, £, C$ → €),
      - считает «цена за м²»,
      - нумерует строки и переименовывает колонки.
    """
    df = df.copy()
    df["info"] = df["info"].astype(str)

    # 1) Убираем жилые комплексы (оставляем индивидуальные объявления)
    #   - вырезаем хвост «Всего ... квартир»
    df["info"] = df["info"].str.replace(r"Всего.*?квартир", "", case=False, regex=True)
    #   - исключаем те, где встречается «Срок сдачи» (признак комплекса)
    mask = ~df["info"].str.contains("Срок сдачи", case=False, regex=True)
    df = df[mask].copy()

    # чистим пустое
    df["info"] = df["info"].str.strip()
    df = df[df["info"] != ""].copy()

    # 2) Очищаем текст info от повторяющихся/мусорных кусочков
    df["info"] = df["info"].str.replace("м?", " ", regex=False)  # иногда встречается "м?"
    for pat in ["площадь", "общая", "зеи:", "\xa0"]:
        df["info"] = df["info"].str.replace(pat, "", regex=False)
    df["info"] = df["info"].str.replace(r"\s+", " ", regex=True).str.strip()

    # извлекаем площадь по шаблону «число + м»
    def extract_square(txt: str):
        match = re.search(r"([\d.,]+)\s*м", txt)
        if match:
            val = match.group(1).replace(",", ".")
            try:
                return float(val)
            except:
                return np.nan
        return np.nan

    df["square"] = df["info"].apply(extract_square)

    # извлекаем кол-во комнат по слову «спальня/спален»
    def extract_rooms(txt: str):
        match = re.search(r"(\d+)\s*спал", txt, re.IGNORECASE)
        if match:
            return float(match.group(1))
        return np.nan

    df["rooms"] = df["info"].apply(extract_rooms)

    # 3) Приводим price к числу и конвертируем в евро по фиксированным коэффициентам
    RATES = {
        "€": 1.0,   # евро — базовая валюта
        "$": 0.96,  # доллар
        "£": 1.20,  # фунт
        "C$": 0.67  # канадский доллар
    }

    def clean_price(p: str):
        """
        Чистим цену:
          - убираем префиксы «От/от», неразрывные пробелы,
          - вытаскиваем первый валидный фрагмент 'C$ 609 000' / '$609,000' / '609 000 €',
          - определяем символ валюты спереди/сзади,
          - нормализуем числовую часть и приводим к float,
          - конвертируем в евро по RATES.
        """
        if not isinstance(p, str):
            return np.nan

        p = p.replace("От", "").replace("от", "").replace("\xa0", " ").strip()

        # На случай дублирования валюты/числа (например, 'C$609 000C$609 000') берём первое вхождение
        # Шаблон валюты: (C\$|\$|£)? ... (€, $, £, C$)?
        match = re.search(r"(C\$|\$|£)?[\d.,\s]+(€|\$|£|C\$)?", p)
        if not match:
            return np.nan

        chunk = match.group(0)

        # Выделяем валюту спереди/сзади и отделяем числовую часть
        currency_front = None
        currency_back = None
        numeric_part = chunk.strip()

        front_match = re.match(r"(C\$|\$|£)", numeric_part)
        if front_match:
            currency_front = front_match.group(1)
            numeric_part = numeric_part[len(currency_front):]

        numeric_part = numeric_part.strip()

        back_match = re.search(r"(C\$|\$|£|€)$", numeric_part)
        if back_match:
            currency_back = back_match.group(1)
            numeric_part = numeric_part[: -len(currency_back)]

        numeric_part = numeric_part.strip()

        # Финальная валюта
        if currency_front:
            currency_symbol = currency_front
        elif currency_back:
            currency_symbol = currency_back
        else:
            currency_symbol = "€"  # по умолчанию евро

        # Чистим числовую часть (оставляем цифры, . и ,)
        numeric_part = re.sub(r"[^0-9.,]", "", numeric_part)
        numeric_part = numeric_part.replace(",", ".").replace(" ", "").strip()
        if not numeric_part:
            return np.nan

        try:
            val = float(numeric_part)
        except:
            return np.nan

        rate = RATES.get(currency_symbol, 1.0)
        return val * rate  # в евро

    df["price"] = df["price"].apply(clean_price)
    df = df.dropna(subset=["price"])

    # 4) Цена за м² (если есть валидная площадь)
    df["price_per_sq"] = np.where(
        (df["price"].notna()) & (df["square"] > 0),
        df["price"] / df["square"],
        np.nan
    )

    # 5) Сквозная нумерация строк
    df.insert(0, "№", range(1, len(df) + 1))

    # Переименуем колонки под общий формат
    rename_map = {
        "name": "Название",
        "price": "Цена",
        "square": "Площадь (м²)",
        "rooms": "Комнаты",
        "price_per_sq": "Цена за м²",
    }
    df = df.rename(columns=rename_map)

    # Оставим только нужные столбцы (если каких-то нет — пропускаем)
    desired_cols = ["№", "Название", "Цена", "Площадь (м²)", "Комнаты", "Цена за м²"]
    existing = [c for c in desired_cols if c in df.columns]
    df = df[existing]

    return df


# ---------- 3) Сохранение в Excel (таблица + мини-сводная) ----------

def save_tranio_excel(df: pd.DataFrame, file_path: str) -> None:
    """
    Сохраняет таблицу в Excel с «умной таблицей»:
      - заголовки на второй строке,
      - данные с третьей,
      - итоги (СРЕДНЕЕ) для «Цена» и «Цена за м²»,
      - мини-сводная в H1..I2.
    """
    wb = Workbook()
    ws = wb.active
    ws.title = "Data"

    if df.empty:
        df.to_excel(file_path, index=False)
        wb.save(file_path)
        return

    cols = df.columns.to_list()
    nrows = len(df)
    ncols = len(cols)

    # заголовки → row=2
    header_row = 2
    for j, c_name in enumerate(cols):
        ws.cell(row=header_row, column=j + 1, value=c_name)

    # данные → row=3..(2 + nrows)
    data_start = 3
    for i in range(nrows):
        for j, c_name in enumerate(cols):
            ws.cell(row=data_start + i, column=j + 1, value=df.iat[i, j])

    data_end_row = data_start + nrows - 1  # = 2 + nrows
    ref_range = f"A{header_row}:{get_column_letter(ncols)}{data_end_row}"

    # умная таблица
    tab = Table(displayName="Table1", ref=ref_range)
    style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True, showColumnStripes=False)
    tab.tableStyleInfo = style
    tab.totalsRowShown = True

    # «СРЕДНЕЕ:» в «Название», average для «Цена» и «Цена за м²»
    for i, colObj in enumerate(tab.tableColumns):
        hdr = cols[i]
        if hdr == "Название":
            colObj.totalsRowLabel = "СРЕДНЕЕ:"
            colObj.totalsRowFunction = None
        elif hdr in ["Цена", "Цена за м²"]:
            colObj.totalsRowFunction = "average"
        else:
            colObj.totalsRowFunction = None

    ws.add_table(tab)

    # форматирование столбцов
    for j, c_name in enumerate(cols, 1):
        ws.column_dimensions[get_column_letter(j)].width = 15
        if c_name in ["Цена", "Цена за м²"]:
            for row_i in range(data_start, data_end_row + 1):
                ws.cell(row=row_i, column=j).number_format = '#,##0.00 "€"'

    # мини-сводная → H1..I2
    ws["H1"] = "Средняя цена"
    ws["I1"] = "Средняя цена за м²"
    ws["H2"] = f"=SUBTOTAL(101,Table1[[#All],[Цена]])"
    ws["I2"] = f"=SUBTOTAL(101,Table1[[#All],[Цена за м²]])"
    ws["H2"].number_format = '#,##0.00 "€"'
    ws["I2"].number_format = '#,##0.00 "€"'
    ws.column_dimensions["H"].width = 20
    ws.column_dimensions["I"].width = 20

    wb.save(file_path)


# ---------- 4) Пример батч-запуска (только Tranio) ----------

def run_tranio_batch():
    """
    Прогон по набору стран (Tranio) с сохранением отдельных Excel-файлов.
    """
    countries = [
        ("Австралия",       "australia"),
        ("Австрия",         "austria"),
        ("Андорра",         "andorra"),
        ("Болгария",        "bulgaria"),
        ("Великобритания",  "united-kingdom"),
        ("Венгрия",         "hungary"),
        ("Германия",        "germany"),
        ("Греция",          "greece"),
        ("Грузия",          "georgia"),
        ("Израиль",         "israel"),
        ("Испания",         "spain"),
        ("Италия",          "italy"),
        ("Канада",          "canada"),
        ("Кипр",            "cyprus"),
        ("Латвия",          "latvia"),
        ("Люксембург",      "luxembourg"),
        ("Монако",          "monaco"),
        ("ОАЭ",             "uae"),
        ("Португалия",      "portugal"),
        ("Словакия",        "slovakia"),
        ("США",             "usa"),
        ("Таиланд",         "thailand"),
        ("Турция",          "turkey"),
        ("Финляндия",       "finland"),
        ("Франция",         "france"),
        ("Хорватия",        "croatia"),
        ("Черногория",      "montenegro"),
        ("Чехия",           "czech-republic"),
        ("Швейцария",       "switzerland"),
        ("Швеция",          "sweden"),
        ("Эстония",         "estonia"),
    ]
    base_path = r"C:\Users\Алина\Desktop\Недвижимость (Tranio)"

    for (country_rus, url_part) in countries:
        print(f"\n=== Tranio: {country_rus} => {url_part} ===")
        df_raw = parse_tranio_country(url_part, max_pages=20)
        df_clean = transform_tranio_data(df_raw)

        file_path = rf"{base_path}\{country_rus}.xlsx"
        save_tranio_excel(df_clean, file_path)
        print(f"[{country_rus}] => {file_path}")


if __name__ == "__main__":
    run_tranio_batch()


=== Tranio: Австралия => australia ===
[Tranio/australia] Парсим страницу 1: https://tranio.ru/australia/apartments/?order=rank
Нет объявлений на странице. Останавливаемся.
[Австралия] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Австралия.xlsx

=== Tranio: Австрия => austria ===
[Tranio/austria] Парсим страницу 1: https://tranio.ru/austria/apartments/?order=rank
[Tranio/austria] Парсим страницу 2: https://tranio.ru/austria/apartments/?order=rank&page=2
[Tranio/austria] Парсим страницу 3: https://tranio.ru/austria/apartments/?order=rank&page=3
[Tranio/austria] Парсим страницу 4: https://tranio.ru/austria/apartments/?order=rank&page=4
Страница 4 полностью дублирует предыдущие. Остановка.
[Австрия] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Австрия.xlsx

=== Tranio: Андорра => andorra ===
[Tranio/andorra] Парсим страницу 1: https://tranio.ru/andorra/apartments/?order=rank
[Tranio/andorra] Парсим страницу 2: https://tranio.ru/andorra/apartments/?order=rank&page=2
Страница 2 полно

[Tranio/greece] Парсим страницу 11: https://tranio.ru/greece/apartments/?order=rank&page=11
[Tranio/greece] Парсим страницу 12: https://tranio.ru/greece/apartments/?order=rank&page=12
[Tranio/greece] Парсим страницу 13: https://tranio.ru/greece/apartments/?order=rank&page=13
[Tranio/greece] Парсим страницу 14: https://tranio.ru/greece/apartments/?order=rank&page=14
[Tranio/greece] Парсим страницу 15: https://tranio.ru/greece/apartments/?order=rank&page=15
[Tranio/greece] Парсим страницу 16: https://tranio.ru/greece/apartments/?order=rank&page=16
[Tranio/greece] Парсим страницу 17: https://tranio.ru/greece/apartments/?order=rank&page=17
[Tranio/greece] Парсим страницу 18: https://tranio.ru/greece/apartments/?order=rank&page=18
[Tranio/greece] Парсим страницу 19: https://tranio.ru/greece/apartments/?order=rank&page=19
Нет объявлений на странице. Останавливаемся.
[Греция] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Греция.xlsx

=== Tranio: Грузия => georgia ===
[Tranio/georgia] Парсим

[Tranio/italy] Парсим страницу 17: https://tranio.ru/italy/apartments/?order=rank&page=17
[Tranio/italy] Парсим страницу 18: https://tranio.ru/italy/apartments/?order=rank&page=18
[Tranio/italy] Парсим страницу 19: https://tranio.ru/italy/apartments/?order=rank&page=19
[Tranio/italy] Парсим страницу 20: https://tranio.ru/italy/apartments/?order=rank&page=20
Достигнут лимит max_pages=20. Остановка.
[Италия] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Италия.xlsx

=== Tranio: Канада => canada ===
[Tranio/canada] Парсим страницу 1: https://tranio.ru/canada/apartments/?order=rank
[Tranio/canada] Парсим страницу 2: https://tranio.ru/canada/apartments/?order=rank&page=2
[Tranio/canada] Парсим страницу 3: https://tranio.ru/canada/apartments/?order=rank&page=3
[Tranio/canada] Парсим страницу 4: https://tranio.ru/canada/apartments/?order=rank&page=4
[Tranio/canada] Парсим страницу 5: https://tranio.ru/canada/apartments/?order=rank&page=5
[Tranio/canada] Парсим страницу 6: https://tranio.ru/

[Tranio/uae] Парсим страницу 16: https://tranio.ru/uae/apartments/?order=rank&page=16
[Tranio/uae] Парсим страницу 17: https://tranio.ru/uae/apartments/?order=rank&page=17
[Tranio/uae] Парсим страницу 18: https://tranio.ru/uae/apartments/?order=rank&page=18
[Tranio/uae] Парсим страницу 19: https://tranio.ru/uae/apartments/?order=rank&page=19
[Tranio/uae] Парсим страницу 20: https://tranio.ru/uae/apartments/?order=rank&page=20
Достигнут лимит max_pages=20. Остановка.
[ОАЭ] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\ОАЭ.xlsx

=== Tranio: Португалия => portugal ===
[Tranio/portugal] Парсим страницу 1: https://tranio.ru/portugal/apartments/?order=rank
[Tranio/portugal] Парсим страницу 2: https://tranio.ru/portugal/apartments/?order=rank&page=2
[Tranio/portugal] Парсим страницу 3: https://tranio.ru/portugal/apartments/?order=rank&page=3
[Tranio/portugal] Парсим страницу 4: https://tranio.ru/portugal/apartments/?order=rank&page=4
[Tranio/portugal] Парсим страницу 5: https://tranio.ru/po

[Tranio/turkey] Парсим страницу 15: https://tranio.ru/turkey/apartments/?order=rank&page=15
[Tranio/turkey] Парсим страницу 16: https://tranio.ru/turkey/apartments/?order=rank&page=16
[Tranio/turkey] Парсим страницу 17: https://tranio.ru/turkey/apartments/?order=rank&page=17
[Tranio/turkey] Парсим страницу 18: https://tranio.ru/turkey/apartments/?order=rank&page=18
[Tranio/turkey] Парсим страницу 19: https://tranio.ru/turkey/apartments/?order=rank&page=19
[Tranio/turkey] Парсим страницу 20: https://tranio.ru/turkey/apartments/?order=rank&page=20
Достигнут лимит max_pages=20. Остановка.
[Турция] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Турция.xlsx

=== Tranio: Финляндия => finland ===
[Tranio/finland] Парсим страницу 1: https://tranio.ru/finland/apartments/?order=rank
[Tranio/finland] Парсим страницу 2: https://tranio.ru/finland/apartments/?order=rank&page=2
Страница 2 полностью дублирует предыдущие. Остановка.
[Финляндия] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Финляндия

[Tranio/czech-republic] Парсим страницу 11: https://tranio.ru/czech-republic/apartments/?order=rank&page=11
[Tranio/czech-republic] Парсим страницу 12: https://tranio.ru/czech-republic/apartments/?order=rank&page=12
[Tranio/czech-republic] Парсим страницу 13: https://tranio.ru/czech-republic/apartments/?order=rank&page=13
Страница 13 полностью дублирует предыдущие. Остановка.
[Чехия] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Чехия.xlsx

=== Tranio: Швейцария => switzerland ===
[Tranio/switzerland] Парсим страницу 1: https://tranio.ru/switzerland/apartments/?order=rank
[Tranio/switzerland] Парсим страницу 2: https://tranio.ru/switzerland/apartments/?order=rank&page=2
Страница 2 полностью дублирует предыдущие. Остановка.
[Швейцария] => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Швейцария.xlsx

=== Tranio: Швеция => sweden ===
[Tranio/sweden] Парсим страницу 1: https://tranio.ru/sweden/apartments/?order=rank
Нет объявлений на странице. Останавливаемся.
[Швеция] => C:\Users\Алина\D

In [4]:
# ---------- 1) Сбор (парсинг страниц) с Prian ----------

def extract_data(url):
    """
    Загружает HTML одной страницы Prian и вытаскивает списки:
      - названия (name),
      - цены (price),
      - площадь в м² (square),
      - количество комнат (rooms).

    Возвращает кортеж из 4 списков одинаковой длины.
    Обработка ошибок:
      - при HTTP 429 (слишком много запросов) — пауза 30 сек и повтор;
      - при любом не-200 статусе — страница пропускается (возвращаются пустые списки).
    """
    HEADERS = {
        # Мнимый «браузерный» заголовок — помогает снизить шанс бана
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/110.0.0.0 Safari/537.36"
        )
    }

    resp = requests.get(url, headers=HEADERS)
    if resp.status_code == 429:
        # Защита от Rate Limit: ждём и пробуем ещё раз
        print(f"[extract_data] HTTP 429 => Ждём 30 сек и повторяем: {url}")
        time.sleep(30)
        resp = requests.get(url, headers=HEADERS)
        if resp.status_code == 429:
            # Если снова 429 — пропускаем страницу
            print(f"[extract_data] Снова 429. Пропускаем страницу: {url}")
            return [], [], [], []
    elif resp.status_code != 200:
        # Любой другой не-успешный статус — пропускаем страницу
        print(f"[extract_data] HTTP {resp.status_code}. Пропускаем: {url}")
        return [], [], [], []

    soup = LxmlSoup(resp.text)

    # --- Ищем элементы карточек ---
    # Название объекта
    names  = soup.find_all("div", class_="b-title")
    # Цена
    prices = soup.find_all("div", class_="price")
    # Блок с пиктограммами (площадь, комнаты и т.п.)
    icons  = soup.find_all("div", class_="b-icon")

    # Подготовим выходные списки
    l_name   = []
    l_price  = []
    l_square = []
    l_rooms  = []

    # Делаем безопасную итерацию по минимальной длине (чтобы не вывалиться по индексу)
    length = min(len(names), len(prices), len(icons))
    for i in range(length):
        name_el  = names[i]
        price_el = prices[i]
        icon_el  = icons[i]

        # Текстовые значения с защитой от None
        name_str  = name_el.text().strip()  if name_el  else "No name"
        price_str = price_el.text().strip() if price_el else "No price"

        # --- Вытаскиваем площадь и кол-во комнат из <span> внутри блока иконок ---
        # На Prian внутри 'b-icon' есть <span> с маркерами '#sheme-square' и '#sheme-room'
        spans = icon_el.find_all("span")
        sq_val = np.nan
        rm_val = np.nan
        for sp in spans:
            # вместо sp.html(), используем str(sp) — так проще проверять подстроки
            sp_str  = str(sp)
            sp_text = sp.text().strip()  # например, "39 м2" или "1"

            # Если это «площадь» — вытаскиваем число
            if "#sheme-square" in sp_str:
                nums = re.findall(r"\d+", sp_text)
                if nums:
                    try:
                        sq_val = float(nums[0])
                    except:
                        sq_val = np.nan

            # Если это «комнаты» — вытаскиваем число
            elif "#sheme-room" in sp_str:
                nums = re.findall(r"\d+", sp_text)
                if nums:
                    try:
                        rm_val = float(nums[0])
                    except:
                        rm_val = np.nan

        # Сохраняем извлечённые поля
        l_name.append(name_str)
        l_price.append(price_str)
        l_square.append(sq_val)
        l_rooms.append(rm_val)

    return l_name, l_price, l_square, l_rooms


def parse_prian_country(country_url_part: str, max_pages=20, step=30):
    """
    Пагинация Prian по параметру next:
      - первая страница:   /{country}/apartments/
      - последующие:       /{country}/apartments/?next={offset}
    где offset увеличивается фиксированным шагом (step), например 0, 30, 60, ...

    Стратегия:
      - идём по страницам, аккумулируем объявления в множесто `unique_entries`,
        чтобы не собирать дубли;
      - останавливаемся, если страница не принесла новых объявлений
        или достигнут лимит max_pages.

    Возвращает DataFrame с сырыми колонками: name, price, square, rooms.
    """
    unique_entries = set()     # для фильтрации дублей
    all_names   = []
    all_prices  = []
    all_squares = []
    all_rooms   = []

    offset = 0          # смещение пагинации (?next=offset)
    page_count = 1      # счётчик страниц (для логов)

    while True:
        # Формируем URL: без next для первой страницы, далее — с ?next={offset}
        if offset == 0:
            url = f"https://prian.ru/{country_url_part}/apartments/"
        else:
            url = f"https://prian.ru/{country_url_part}/apartments/?next={offset}"

        print(f"[parse_prian_country] page={page_count}, offset={offset}: {url}")
        # Небольшая случайная пауза — вежливость к сайту
        time.sleep(random.uniform(1, 2))

        # Забираем по текущей странице 4 параллельных списка полей
        names, prices, squares, rooms = extract_data(url)
        length = min(len(names), len(prices), len(squares), len(rooms))

        # Если ничего не нашли — заканчиваем
        if length == 0:
            print("Нет (новых) объявлений. Остановка.")
            break

        found_new = False
        for i in range(length):
            entry = (names[i], prices[i], squares[i], rooms[i])
            if entry not in unique_entries:
                # Сохраняем только новые объявления
                unique_entries.add(entry)
                all_names.append(names[i])
                all_prices.append(prices[i])
                all_squares.append(squares[i])
                all_rooms.append(rooms[i])
                found_new = True

        # Если страница полностью дублирует предыдущие — дальше смысла нет
        if not found_new:
            print("Все объявления этой страницы уже были. Остановка.")
            break

        # Готовим следующую страницу
        offset += step      # на Prian смещение увеличивается фиксированно
        page_count += 1
        if page_count > max_pages:
            print(f"max_pages={max_pages} достигнут. Остановка.")
            break

    # Складываем всё в DataFrame «сырых» данных
    df = pd.DataFrame({
        "name":   all_names,
        "price":  all_prices,
        "square": all_squares,
        "rooms":  all_rooms
    })
    return df


# ---------- 2) Преобразование (очистка и расчёты) ----------

def transform_prian_data(df: pd.DataFrame, country_rus: str) -> pd.DataFrame:
    """
    Приводит «сырые» данные Prian к единому виду:
      1) фильтрует по стране (в name должен быть суффикс ', {Страна}'),
      2) чистит цену и удаляет NaN,
      3) убирает нежелательные типы (пример: 'Земля в Пафосе, Кипр', 'Магазин'),
      4) считает «цена за м²»,
      5) нумерует строки и переименовывает колонки в общий формат.

    На случай полного «пусто» всегда возвращает DataFrame с нужными колонками.
    """
    # Подготовим «пустышку» на случай отсутствия данных
    columns_needed = ["№","Название","Цена","Площадь (м²)","Комнаты","Цена за м²"]
    df_final = pd.DataFrame(columns=columns_needed)

    if df.empty:
        print("[transform_prian_data] DF пуст => возвращаем пустую таблицу.")
        return df_final

    df = df.copy()

    # (1) Фильтр по стране: в Prian в конце name обычно есть ', {Страна}'
    suffix = ", " + country_rus
    mask = df["name"].str.endswith(suffix, na=False)
    df = df[mask].copy()
    if df.empty:
        print("[transform_prian_data] После фильтрации по стране => пусто.")
        return df_final

    # (2) Чистка цены: убираем 'от ', евро-символ, неразрывные пробелы и обычные пробелы
    #     Преобразуем строку в float (если 'ценапозапросу' — NaN)
    def clean_price(p: str):
        if not isinstance(p, str):
            return np.nan
        s = p.replace("от ","").replace("€","").replace("\xa0","").replace(" ","")
        if s.lower() == "ценапозапросу":
            return np.nan
        try:
            return float(s)
        except:
            return np.nan

    df["price"] = df["price"].apply(clean_price)
    df = df.dropna(subset=["price"])
    if df.empty:
        print("[transform_prian_data] Все price=NaN => пусто.")
        return df_final

    # (3) Удаляем нежелательные типы объектов
    bad_vals = ["Земля в Пафосе, Кипр", "Магазин"]
    for val in bad_vals:
        mask_n = df["name"].str.contains(val, case=False, na=False)
        df = df[~mask_n].copy()
    if df.empty:
        print("[transform_prian_data] После удаления bad_vals => пусто.")
        return df_final

    # (4) Цена за м² (только если есть валидная площадь)
    def calc_pps(r):
        pr = r["price"]
        sq = r["square"]
        if pd.isna(pr) or pd.isna(sq) or sq == 0:
            return np.nan
        return pr / sq

    df["price_per_sq"] = df.apply(calc_pps, axis=1)

    # (5) Нумерация строк + финальные имена колонок
    df.insert(0, "№", range(1, len(df) + 1))

    rename_map = {
        "name": "Название",
        "price": "Цена",
        "square": "Площадь (м²)",
        "rooms": "Комнаты",
        "price_per_sq": "Цена за м²"
    }
    df = df.rename(columns=rename_map)

    # Оставляем только нужные столбцы (в нужном порядке)
    df = df[["№","Название","Цена","Площадь (м²)","Комнаты","Цена за м²"]]

    # На случай, если после всех фильтров внезапно пусто — возвращаем пустышку
    if df.empty:
        return df_final

    return df


# ---------- 3) Сохранение в Excel (таблица + мини-сводная) ----------

def save_prian_excel(df: pd.DataFrame, file_path: str):
    """
    Сохраняет таблицу в Excel с «умной» таблицей:
      - заголовки на второй строке,
      - данные с третьей,
      - итоги 'СРЕДНЕЕ' для 'Цена' и 'Цена за м²',
      - мини-сводная в H1..I2.
    Если df пуст — сохраняет плоский Excel (для единообразия пайплайна).
    """
    wb = Workbook()
    ws = wb.active
    ws.title = "Data"

    if df.empty:
        df.to_excel(file_path, index=False)
        return

    cols = df.columns.tolist()
    nrows = len(df)
    ncols = len(cols)

    # Заголовки => row=2
    header_row = 2
    for j, c_name in enumerate(cols):
        ws.cell(row=header_row, column=j + 1, value=c_name)

    # Данные => row=3..(2 + nrows)
    data_start = 3
    for i in range(nrows):
        for j, c_name in enumerate(cols):
            ws.cell(row=data_start + i, column=j + 1, value=df.iat[i, j])

    data_end_row = data_start + nrows - 1  # = 2 + nrows
    ref_range = f"A{header_row}:{get_column_letter(ncols)}{data_end_row}"

    # Умная таблица с подсветкой полос
    tab = Table(displayName="Table1", ref=ref_range)
    style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True, showColumnStripes=False)
    tab.tableStyleInfo = style
    tab.totalsRowShown = True

    # Итоги: «СРЕДНЕЕ:» — в колонке «Название», average — в «Цена» и «Цена за м²»
    for i, colObj in enumerate(tab.tableColumns):
        hdr = cols[i]
        if hdr == "Название":
            colObj.totalsRowLabel = "СРЕДНЕЕ:"
            colObj.totalsRowFunction = None
        elif hdr in ["Цена", "Цена за м²"]:
            colObj.totalsRowFunction = "average"
        else:
            colObj.totalsRowFunction = None

    ws.add_table(tab)

    # Немного форматирования
    for j, c_name in enumerate(cols, 1):
        ws.column_dimensions[get_column_letter(j)].width = 15
        if c_name in ["Цена", "Цена за м²"]:
            for row_i in range(data_start, data_end_row + 1):
                ws.cell(row=row_i, column=j).number_format = '#,##0.00 "€"'

    # Мини-сводная => H1..I2
    ws["H1"] = "Средняя цена"
    ws["I1"] = "Средняя цена за м²"
    ws["H2"] = f"=SUBTOTAL(101,Table1[[#All],[Цена]])"
    ws["I2"] = f"=SUBTOTAL(101,Table1[[#All],[Цена за м²]])"
    ws["H2"].number_format = '#,##0.00 "€"'
    ws["I2"].number_format = '#,##0.00 "€"'
    ws.column_dimensions["H"].width = 20
    ws.column_dimensions["I"].width = 20

    wb.save(file_path)


# ---------- 4) Пример батч-запуска (только Prian) ----------

def main():
    """
    Прогон по списку стран для Prian.
    """
    countries = [
        ("Австралия",        "australia"),
        ("Австрия",          "austria"),
        ("Андорра",          "andorra"),
        ("Болгария",         "bulgaria"),
        ("Венгрия",          "hungary"),
        ("Германия",         "germany"),
        ("Греция",           "greece"),
        ("Грузия",           "georgia"),
        ("Израиль",          "israel"),
        ("Испания",          "spain"),
        ("Италия",           "italy"),
        ("Кипр",             "cyprus"),
        ("Латвия",           "latvia"),
        ("Литва",            "lithuania"),
        ("Люксембург",       "luxembourg"),
        ("Монако",           "monaco"),
        ("Нидерланды",       "netherlands"),
        ("ОАЭ",              "united-arab-emirates"),
        ("Португалия",       "portugal"),
        ("Великобритания",   "great-britain"),
        ("США",              "usa"),
        ("Таиланд",          "thailand"),
        ("Турция",           "turkey"),
        ("Финляндия",        "finland"),
        ("Франция",          "france"),
        ("Хорватия",         "croatia"),
        ("Черногория",       "montenegro"),
        ("Чехия",            "czech"),
        ("Швейцария",        "switzerland"),
        ("Швеция",           "sweden"),
        ("Эстония",          "estonia"),
    ]
    base_path = r"C:\Users\Алина\Desktop\Недвижимость (Prian)"

    for (country_rus, cpart) in countries:
        print(f"\n=== Prian: {country_rus} => {cpart} ===")
        df_raw  = parse_prian_country(cpart, max_pages=20, step=30)
        df_clean = transform_prian_data(df_raw, country_rus)

        file_path = rf"{base_path}\{country_rus}.xlsx"
        save_prian_excel(df_clean, file_path)
        print(f"[{country_rus}] => {file_path}")

    print("\nГотово!")


if __name__ == "__main__":
    main()


=== Prian: Австралия => australia ===
[parse_prian_country] page=1, offset=0: https://prian.ru/australia/apartments/
[parse_prian_country] page=2, offset=30: https://prian.ru/australia/apartments/?next=30
[parse_prian_country] page=3, offset=60: https://prian.ru/australia/apartments/?next=60
Все объявления этой страницы уже были. Остановка.
[transform_prian_data] После фильтрации по стране => пусто.
[Австралия] => C:\Users\Алина\Desktop\Недвижимость (Prian)\Австралия.xlsx

=== Prian: Австрия => austria ===
[parse_prian_country] page=1, offset=0: https://prian.ru/austria/apartments/
[parse_prian_country] page=2, offset=30: https://prian.ru/austria/apartments/?next=30
[parse_prian_country] page=3, offset=60: https://prian.ru/austria/apartments/?next=60
[parse_prian_country] page=4, offset=90: https://prian.ru/austria/apartments/?next=90
[parse_prian_country] page=5, offset=120: https://prian.ru/austria/apartments/?next=120
[parse_prian_country] page=6, offset=150: https://prian.ru/austr

[parse_prian_country] page=10, offset=270: https://prian.ru/georgia/apartments/?next=270
[parse_prian_country] page=11, offset=300: https://prian.ru/georgia/apartments/?next=300
[parse_prian_country] page=12, offset=330: https://prian.ru/georgia/apartments/?next=330
[parse_prian_country] page=13, offset=360: https://prian.ru/georgia/apartments/?next=360
[parse_prian_country] page=14, offset=390: https://prian.ru/georgia/apartments/?next=390
[parse_prian_country] page=15, offset=420: https://prian.ru/georgia/apartments/?next=420
[parse_prian_country] page=16, offset=450: https://prian.ru/georgia/apartments/?next=450
[parse_prian_country] page=17, offset=480: https://prian.ru/georgia/apartments/?next=480
[parse_prian_country] page=18, offset=510: https://prian.ru/georgia/apartments/?next=510
[parse_prian_country] page=19, offset=540: https://prian.ru/georgia/apartments/?next=540
[parse_prian_country] page=20, offset=570: https://prian.ru/georgia/apartments/?next=570
max_pages=20 достигну

Все объявления этой страницы уже были. Остановка.
[Латвия] => C:\Users\Алина\Desktop\Недвижимость (Prian)\Латвия.xlsx

=== Prian: Литва => lithuania ===
[parse_prian_country] page=1, offset=0: https://prian.ru/lithuania/apartments/
[parse_prian_country] page=2, offset=30: https://prian.ru/lithuania/apartments/?next=30
Все объявления этой страницы уже были. Остановка.
[transform_prian_data] После фильтрации по стране => пусто.
[Литва] => C:\Users\Алина\Desktop\Недвижимость (Prian)\Литва.xlsx

=== Prian: Люксембург => luxembourg ===
[parse_prian_country] page=1, offset=0: https://prian.ru/luxembourg/apartments/
[parse_prian_country] page=2, offset=30: https://prian.ru/luxembourg/apartments/?next=30
[parse_prian_country] page=3, offset=60: https://prian.ru/luxembourg/apartments/?next=60
Все объявления этой страницы уже были. Остановка.
[Люксембург] => C:\Users\Алина\Desktop\Недвижимость (Prian)\Люксембург.xlsx

=== Prian: Монако => monaco ===
[parse_prian_country] page=1, offset=0: https:

[parse_prian_country] page=13, offset=360: https://prian.ru/usa/apartments/?next=360
[parse_prian_country] page=14, offset=390: https://prian.ru/usa/apartments/?next=390
[parse_prian_country] page=15, offset=420: https://prian.ru/usa/apartments/?next=420
[parse_prian_country] page=16, offset=450: https://prian.ru/usa/apartments/?next=450
[parse_prian_country] page=17, offset=480: https://prian.ru/usa/apartments/?next=480
[parse_prian_country] page=18, offset=510: https://prian.ru/usa/apartments/?next=510
[parse_prian_country] page=19, offset=540: https://prian.ru/usa/apartments/?next=540
[parse_prian_country] page=20, offset=570: https://prian.ru/usa/apartments/?next=570
max_pages=20 достигнут. Остановка.
[США] => C:\Users\Алина\Desktop\Недвижимость (Prian)\США.xlsx

=== Prian: Таиланд => thailand ===
[parse_prian_country] page=1, offset=0: https://prian.ru/thailand/apartments/
[parse_prian_country] page=2, offset=30: https://prian.ru/thailand/apartments/?next=30
[parse_prian_country] 

max_pages=20 достигнут. Остановка.
[Франция] => C:\Users\Алина\Desktop\Недвижимость (Prian)\Франция.xlsx

=== Prian: Хорватия => croatia ===
[parse_prian_country] page=1, offset=0: https://prian.ru/croatia/apartments/
[parse_prian_country] page=2, offset=30: https://prian.ru/croatia/apartments/?next=30
[parse_prian_country] page=3, offset=60: https://prian.ru/croatia/apartments/?next=60
[parse_prian_country] page=4, offset=90: https://prian.ru/croatia/apartments/?next=90
[parse_prian_country] page=5, offset=120: https://prian.ru/croatia/apartments/?next=120
[parse_prian_country] page=6, offset=150: https://prian.ru/croatia/apartments/?next=150
[parse_prian_country] page=7, offset=180: https://prian.ru/croatia/apartments/?next=180
[parse_prian_country] page=8, offset=210: https://prian.ru/croatia/apartments/?next=210
[parse_prian_country] page=9, offset=240: https://prian.ru/croatia/apartments/?next=240
[parse_prian_country] page=10, offset=270: https://prian.ru/croatia/apartments/?next

In [5]:
# ================================
#  Создание совокупной таблицы
#  (объединяем результаты 3 сайтов
#   в один Excel с вкладкой на страну)
# ================================

# Список стран, для которых есть выгрузки по каждому сайту
COUNTRIES_33 = [
    "Австралия","Австрия","Андорра","Болгария","Венгрия","Германия","Греция","Грузия",
    "Израиль","Испания","Италия","Канада","Кипр","Латвия","Литва","Люксембург",
    "Монако","Нидерланды","ОАЭ","Португалия","Словакия","Великобритания","США","Таиланд",
    "Турция","Финляндия","Франция","Хорватия","Черногория","Чехия","Швейцария","Швеция","Эстония"
]

# Папки, где лежат ИТОГОВЫЕ XLSX от каждого отдельного парсера
HOMESOVERSEAS_DIR = r"C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)"
PRIAN_DIR          = r"C:\Users\Алина\Desktop\Недвижимость (Prian)"
TRANIO_DIR         = r"C:\Users\Алина\Desktop\Недвижимость (Tranio)"

# Куда сохранить общий файл
OUTPUT_EXCEL       = r"C:\Users\Алина\Desktop\Недвижимость.xlsx"


# ---------- Вспомогательные функции ----------

def load_data_only(site_dir: str, country: str, site_name: str) -> pd.DataFrame:
    """
    Читает файл <site_dir>/<country>.xlsx, вытаскивает только «плоские» данные
    (без формул/сводных) и добавляет колонку «Сайт».

    Ожидается, что внутри XLSX:
      - заголовки таблицы начинаются со 2-й строки (header=1 в pandas.read_excel),
      - столбцы: №, Название, Цена, Площадь (м²), Комнаты, Цена за м².

    Если файл отсутствует или не читается — возвращается ПУСТОЙ DataFrame
    с нужными колонками (для корректной конкатенации дальше).
    """
    needed = ["№","Название","Цена","Площадь (м²)","Комнаты","Цена за м²","Сайт"]
    path = os.path.join(site_dir, f"{country}.xlsx")
    if not os.path.isfile(path):
        print(f"[load_data_only] {site_name}: Нет файла: {path} => пуст.")
        return pd.DataFrame(columns=needed)

    print(f"[load_data_only] {site_name}, {country} => {path}")
    try:
        # header=1 -> использовать вторую строку как заголовок (A2:F2)
        df0 = pd.read_excel(path, header=1)
    except Exception as e:
        print(f"  Ошибка чтения: {e}")
        return pd.DataFrame(columns=needed)

    # Гарантируем наличие всех базовых столбцов (если чего-то не хватает — добавим пустое)
    base_cols = ["№","Название","Цена","Площадь (м²)","Комнаты","Цена за м²"]
    for col in base_cols:
        if col not in df0.columns:
            df0[col] = np.nan

    # Берём только нужные столбцы и выкидываем полностью пустые строки
    df = df0[base_cols].copy()
    df = df.dropna(how="all")

    # Добавляем признак источника
    df["Сайт"] = site_name

    # Финальный порядок колонок
    df = df[["№","Название","Цена","Площадь (м²)","Комнаты","Цена за м²","Сайт"]]
    print(f"  shape={df.shape}")
    return df


def write_df_as_table(
    ws,
    df: pd.DataFrame,
    start_col: int,
    start_row: int,
    table_name: str,
    table_width: int = 14,
    fill_if_empty=None
):
    """
    Пишет DataFrame (ровно 7 колонок) на лист Excel, начиная с (start_row, start_col),
    создаёт «умную таблицу» (OpenXML Table) с полосами и итоговой строкой.

    Правила итоговой строки:
      - в колонке «Название» отображается ярлык «СРЕДНЕЕ:» (без функции),
      - для «Цена» и «Цена за м²» включается функция average,
      - для остальных колонок итоги отключены.

    Также:
      - задаётся ширина столбцов,
      - для числовых колонок «Цена»/«Цена за м²» выставляется формат '#,##0.00 "€"'.

    Возвращает кортеж (end_row, end_col) — нижний правый угол записанного диапазона.
    """
    nrows = len(df)
    ncols = len(df.columns)

    # Заголовки таблицы
    for j, col_name in enumerate(df.columns):
        ws.cell(row=start_row, column=start_col + j, value=col_name)

    if nrows == 0:
        # Если данных нет — подсветим заголовки (опционально) и вернёмся
        if fill_if_empty:
            fill = PatternFill(start_color=fill_if_empty, end_color=fill_if_empty, fill_type="solid")
            for j in range(ncols):
                ws.cell(row=start_row, column=start_col + j).fill = fill
        end_row = start_row
        end_col = start_col + ncols - 1
    else:
        # Данные под заголовком (начиная со следующей строки)
        for i in range(nrows):
            for j in range(ncols):
                val = df.iat[i, j]
                ws.cell(row=start_row + 1 + i, column=start_col + j, value=val)
        end_row = start_row + nrows
        end_col = start_col + ncols - 1

    # Диапазон «умной таблицы»
    ref = f"{get_column_letter(start_col)}{start_row}:{get_column_letter(end_col)}{end_row}"
    ex_table = Table(displayName=table_name, ref=ref)
    style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True, showColumnStripes=False)
    ex_table.tableStyleInfo = style
    ex_table.totalsRowShown = True  # показываем строку итогов

    # Логика итогов по колонкам
    col_list = df.columns.tolist()
    for i, colObj in enumerate(ex_table.tableColumns):
        hdr = col_list[i]
        if hdr == "Название":
            colObj.totalsRowLabel = "СРЕДНЕЕ:"
            colObj.totalsRowFunction = None
        elif hdr in ["Цена", "Цена за м²"]:
            colObj.totalsRowFunction = "average"
        else:
            colObj.totalsRowFunction = None

    ws.add_table(ex_table)

    # Универсальная ширина столбцов
    for c in range(start_col, start_col + ncols):
        col_letter = get_column_letter(c)
        ws.column_dimensions[col_letter].width = table_width

    # Числовой формат для денежных колонок
    for j, hdr in enumerate(col_list):
        if hdr in ["Цена", "Цена за м²"]:
            col_x = start_col + j
            row_data_begin = start_row + 1
            row_data_end   = end_row
            for r_i in range(row_data_begin, row_data_end + 1):
                ws.cell(row=r_i, column=col_x).number_format = '#,##0.00 "€"'

    return (end_row, end_col)


# ---------- Главная функция объединения ----------

def main():
    """
    Для каждой страны из COUNTRIES_33:
      1) читаем «плоские» таблицы из трёх источников (HomesOverseas/Prian/Tranio),
      2) объединяем их в единую таблицу df_all,
      3) перенумеровываем общий «№»,
      4) записываем на отдельный лист в общий Excel,
      5) добавляем мини-сводную (средняя цена и средняя цена за м²).

    На выходе: один Excel-файл OUTPUT_EXCEL с вкладкой на каждую страну.
    """
    wb = Workbook()
    wb.remove(wb.active)  # удаляем дефолтный пустой лист

    # Стиль/цвета (задаём однажды; RED_FONT/CENTER_AL не используются ниже, но оставляем как в исходнике)
    RED_FONT   = Font(color="FF0000", bold=True)
    CENTER_AL  = Alignment(horizontal="center", vertical="center")
    BLUE_COLOR = "99CCFF"  # этим цветом подсвечиваем заголовок, если данных нет

    for country in COUNTRIES_33:
        print(f"\n=== {country} ===")
        # Название листа максимум 31 символ — обрежем, если что
        ws = wb.create_sheet(title=country[:31])

        # 1) Загружаем данные из трёх источников (каждый даёт 7 колонок + «Сайт»)
        df_homes = load_data_only(HOMESOVERSEAS_DIR, country, "HomesOverseas")
        df_prian = load_data_only(PRIAN_DIR,          country, "Prian")
        df_tran  = load_data_only(TRANIO_DIR,         country, "Tranio")

        # 2) Объединяем в одну таблицу
        df_all = pd.concat([df_homes, df_prian, df_tran], ignore_index=True)
        print(f"[{country}] Объединённый shape={df_all.shape}")

        # 3) Общая перенумерация «№» (сквозная внутри страны)
        if not df_all.empty:
            df_all = df_all.copy()
            df_all["№"] = range(1, len(df_all) + 1)

        # 4) Пишем таблицу на лист:
        #    шапка будет в B8 (т.е. колонка 2, строка 8)
        table_name_all = f"All_{country[:8]}".replace(" ", "_")
        end_row, end_col = write_df_as_table(
            ws, df_all,
            start_col=2,   # колонка B
            start_row=8,   # строка 8
            table_name=table_name_all,
            fill_if_empty=BLUE_COLOR
        )

        # 5) Мини-сводная (B2:C3): средняя цена и средняя цена за м²
        ws["B2"] = "Средняя цена"
        ws["C2"] = "Средняя цена за м²"

        # SUBTOTAL(101, <TableName>[[#All],[Цена]]) — среднее по видимым строкам таблицы
        ws["B3"] = f"=SUBTOTAL(101,{table_name_all}[[#All],[Цена]])"
        ws["C3"] = f"=SUBTOTAL(101,{table_name_all}[[#All],[Цена за м²]])"
        ws["B3"].number_format = '#,##0.00 "€"'
        ws["C3"].number_format = '#,##0.00 "€"'

        # Оформим мини-табличку для сводных значений (полосы как у основной)
        pivot_ref = "B2:C3"
        pivot_table_name = f"Pivot_{country[:8]}".replace(" ", "_")
        pivot_table = Table(displayName=pivot_table_name, ref=pivot_ref)
        pivot_style = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True, showColumnStripes=False)
        pivot_table.tableStyleInfo = pivot_style
        ws.add_table(pivot_table)

        ws.column_dimensions["B"].width = 18
        ws.column_dimensions["C"].width = 20

        # Подбор ширины под колонку «Сайт» (обычно это колонка H в нашей раскладке)
        if not df_all.empty:
            max_len = max(len(str(x)) for x in df_all["Сайт"].dropna())
            desired_width = max(10, min(40, max_len + 5))  # ограничим разумно
        else:
            desired_width = 15
        ws.column_dimensions["H"].width = desired_width

    # Сохраняем общий Excel
    wb.save(OUTPUT_EXCEL)
    print(f"\nИтоговый Excel-файл создан: {OUTPUT_EXCEL}")


if __name__ == "__main__":
    main()


=== Австралия ===
[load_data_only] HomesOverseas, Австралия => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Австралия.xlsx
  shape=(0, 7)
[load_data_only] Prian, Австралия => C:\Users\Алина\Desktop\Недвижимость (Prian)\Австралия.xlsx
  Ошибка чтения: Passed header=[1], len of 1, but only 1 lines in file
[load_data_only] Tranio, Австралия => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Австралия.xlsx
  shape=(0, 7)
[Австралия] Объединённый shape=(0, 7)

=== Австрия ===
[load_data_only] HomesOverseas, Австрия => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Австрия.xlsx
  shape=(89, 7)
[load_data_only] Prian, Австрия => C:\Users\Алина\Desktop\Недвижимость (Prian)\Австрия.xlsx
  shape=(44, 7)
[load_data_only] Tranio, Австрия => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Австрия.xlsx
  shape=(45, 7)
[Австрия] Объединённый shape=(178, 7)

=== Андорра ===
[load_data_only] HomesOverseas, Андорра => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Андорра.xlsx
  shape=(1, 7)

  shape=(74, 7)
[load_data_only] Tranio, Великобритания => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Великобритания.xlsx
  shape=(180, 7)
[Великобритания] Объединённый shape=(315, 7)

=== США ===
[load_data_only] HomesOverseas, США => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\США.xlsx
  shape=(2, 7)
[load_data_only] Prian, США => C:\Users\Алина\Desktop\Недвижимость (Prian)\США.xlsx
  shape=(565, 7)
[load_data_only] Tranio, США => C:\Users\Алина\Desktop\Недвижимость (Tranio)\США.xlsx
  shape=(366, 7)
[США] Объединённый shape=(933, 7)

=== Таиланд ===
[load_data_only] HomesOverseas, Таиланд => C:\Users\Алина\Desktop\Недвижимость (HomesOverseas)\Таиланд.xlsx
  shape=(998, 7)
[load_data_only] Prian, Таиланд => C:\Users\Алина\Desktop\Недвижимость (Prian)\Таиланд.xlsx
  shape=(535, 7)
[load_data_only] Tranio, Таиланд => C:\Users\Алина\Desktop\Недвижимость (Tranio)\Таиланд.xlsx
  shape=(691, 7)
[Таиланд] Объединённый shape=(2224, 7)

=== Турция ===
[load_data_only] HomesOverseas,